<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Guided Practice with Topic Modeling and LDA

_Authors: Dave Yerrington (SF)_

---

> **Note: this lab is intended to be a guided lab with the instructor.**

In practice it would be a very rare to need to build an unsupervised topic model like LDA from scratch. Lucky for us, sklearn comes with LDA topic modeling functionality. Another popular LDA module which we will explore in this lab is from the `gensim` package. 

Let's explore a brief walkthrough of LDA and topic modeling using gensim. We will work with a small collection of documents represented as a list.

### 1. Load the packages and create the small "documents".

You may need to install the gensim package with `pip` or `conda`.

In [1]:
from gensim import corpora, models, matutils
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from collections import defaultdict
import pandas as pd


doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."

# compile sample documents into a list
documents = [doc_a, doc_b, doc_c, doc_d, doc_e]
# convert list into DataFrame
df        = pd.DataFrame(documents, columns=['text'])

In [2]:
df

,text
0,Brocolli is good to eat. My brother likes to e...
1,My mother spends a lot of time driving my brot...
2,Some health experts suggest that driving may c...
3,I often feel pressure to perform well at schoo...
4,Health professionals say that brocolli is good...


### 2. Load stop words either from NLTK or sklearn

In [3]:
from nltk.corpus import stopwords
#from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [4]:
# Prepare the list of stop words from NLTK
stop_words = stopwords.words('english')

# If you need to extend the list of stop words whilst testing
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [5]:
len(stop_words)

179

### 3. Use CountVectorizer to transform our text, taking out the stopwords.

In [6]:
vect = CountVectorizer(stop_words=stop_words)

X_vect = vect.fit_transform(df.text)

In [9]:
print(X_vect)

  (0, 4)	2
  (0, 12)	2
  (0, 9)	2
  (0, 5)	1
  (0, 15)	1
  (0, 18)	1
  (1, 5)	1
  (1, 18)	1
  (1, 28)	1
  (1, 16)	1
  (1, 31)	1
  (1, 8)	1
  (1, 0)	1
  (1, 1)	1
  (1, 22)	1
  (2, 8)	1
  (2, 13)	1
  (2, 10)	1
  (2, 29)	1
  (2, 17)	1
  (2, 6)	1
  (2, 14)	1
  (2, 30)	1
  (2, 3)	1
  (2, 23)	1
  (3, 5)	1
  (3, 18)	1
  (3, 23)	1
  (3, 20)	1
  (3, 11)	1
  (3, 21)	1
  (3, 32)	1
  (3, 26)	1
  (3, 19)	1
  (3, 27)	1
  (3, 7)	1
  (3, 2)	1
  (4, 4)	1
  (4, 12)	1
  (4, 13)	2
  (4, 24)	1
  (4, 25)	1


### 4. Extract the tokens that remain after stopword removal.

The `.vocabulary_` attribute of the vectorizer contains a dictionary of terms. There is also the built-in function `.get_feature_names()` which will extract the column names.

In [10]:
# 33 words
len(vect.vocabulary_)

33

In [11]:
# 33 features
vect.get_feature_names_out()

array(['around', 'baseball', 'better', 'blood', 'brocolli', 'brother',
       'cause', 'drive', 'driving', 'eat', 'experts', 'feel', 'good',
       'health', 'increased', 'likes', 'lot', 'may', 'mother', 'never',
       'often', 'perform', 'practice', 'pressure', 'professionals', 'say',
       'school', 'seems', 'spends', 'suggest', 'tension', 'time', 'well'],
      dtype=object)

### 5. Get counts of tokens.

Convert the matrix from the vectorizer to a dense matrix, then sum by column to get the counts per term.

In [12]:
# Convert sparse matrix to dense matrix
X_array = X_vect.todense()

In [13]:
pd.DataFrame(X_array.sum(axis=0), columns=vect.get_feature_names_out())

,around,baseball,better,blood,brocolli,brother,cause,drive,driving,eat,...,pressure,professionals,say,school,seems,spends,suggest,tension,time,well
0,1,1,1,1,3,3,1,1,2,2,...,2,1,1,1,1,1,1,1,1,1


In [14]:
X_array.sum(axis=0)

matrix([[1, 1, 1, 1, 3, 3, 1, 1, 2, 2, 1, 1, 3, 3, 1, 1, 1, 1, 3, 1, 1,
         1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int64)

In [ ]:
# 8 words appear more than once

### 6. Setup the vocabulary dictionary

First we need to setup the vocabulary.  Gensim's LDA expects our vocabulary to be in a format where the dictionary keys are the column indices and the values are the words themselves.

Create this dictionary below.

In [15]:
# Access column indices
vect.vocabulary_.values()

dict_values([4, 12, 9, 5, 15, 18, 28, 16, 31, 8, 0, 1, 22, 13, 10, 29, 17, 6, 14, 30, 3, 23, 20, 11, 21, 32, 26, 19, 27, 7, 2, 24, 25])

In [16]:
# Access values (words) 
vect.vocabulary_.keys()

dict_keys(['brocolli', 'good', 'eat', 'brother', 'likes', 'mother', 'spends', 'lot', 'time', 'driving', 'around', 'baseball', 'practice', 'health', 'experts', 'suggest', 'may', 'cause', 'increased', 'tension', 'blood', 'pressure', 'often', 'feel', 'perform', 'well', 'school', 'never', 'seems', 'drive', 'better', 'professionals', 'say'])

In [17]:
# Create dictionary where key names are indices and values are the words
lda_dict = dict(zip(vect.vocabulary_.values(), vect.vocabulary_.keys()))

In [18]:
# Remember dictionary is unordered
len(lda_dict)

33

In [19]:
lda_dict

{4: 'brocolli',
 12: 'good',
 9: 'eat',
 5: 'brother',
 15: 'likes',
 18: 'mother',
 28: 'spends',
 16: 'lot',
 31: 'time',
 8: 'driving',
 0: 'around',
 1: 'baseball',
 22: 'practice',
 13: 'health',
 10: 'experts',
 29: 'suggest',
 17: 'may',
 6: 'cause',
 14: 'increased',
 30: 'tension',
 3: 'blood',
 23: 'pressure',
 20: 'often',
 11: 'feel',
 21: 'perform',
 32: 'well',
 26: 'school',
 19: 'never',
 27: 'seems',
 7: 'drive',
 2: 'better',
 24: 'professionals',
 25: 'say'}

### 7. Create a token to id mapping with gensim's `corpora.Dictionary`

This dictionary class is a more standard way to work with with gensim models. There are a few standard steps we should go through:

**7.1. Count the frequency of words.**

We can do this easily with the python `defaultdict(int)`, which doesn't require us to already have the key in the dictionary to be able to add to it:

```python
frequency = defaultdict(int)

for text in documents:
    for token in text.split():
        frequency[token] += 1
```




In [20]:
frequency = defaultdict(int)

for text in documents:
    for token in text.split():
        frequency[token] += 1

In [21]:
print(frequency)

defaultdict(<class 'int'>, {'Brocolli': 1, 'is': 2, 'good': 3, 'to': 6, 'eat.': 1, 'My': 2, 'brother': 3, 'likes': 1, 'eat': 1, 'brocolli,': 1, 'but': 2, 'not': 1, 'my': 4, 'mother.': 1, 'mother': 2, 'spends': 1, 'a': 1, 'lot': 1, 'of': 1, 'time': 1, 'driving': 2, 'around': 1, 'baseball': 1, 'practice.': 1, 'Some': 1, 'health': 1, 'experts': 1, 'suggest': 1, 'that': 2, 'may': 1, 'cause': 1, 'increased': 1, 'tension': 1, 'and': 1, 'blood': 1, 'pressure.': 1, 'I': 1, 'often': 1, 'feel': 1, 'pressure': 1, 'perform': 1, 'well': 1, 'at': 1, 'school,': 1, 'never': 1, 'seems': 1, 'drive': 1, 'do': 1, 'better.': 1, 'Health': 1, 'professionals': 1, 'say': 1, 'brocolli': 1, 'for': 1, 'your': 1, 'health.': 1})


**7.2 Remove any words that only appear once, or appear in the stopwords.**

Iterate through the documents and only keep useful words/tokens.

In [22]:
def remove_words(texts):
    #for key, value in dict(texts).items():
        #if value == 1:
            #del texts[key]
    return [[word for word in doc if word not in stop_words] for doc in texts]

In [23]:
remove_words(frequency)

[['B', 'r', 'c', 'l', 'l'],
 [],
 ['g'],
 [],
 ['e', '.'],
 ['M'],
 ['b', 'r', 'h', 'e', 'r'],
 ['l', 'k', 'e'],
 ['e'],
 ['b', 'r', 'c', 'l', 'l', ','],
 ['b', 'u'],
 ['n'],
 [],
 ['h', 'e', 'r', '.'],
 ['h', 'e', 'r'],
 ['p', 'e', 'n'],
 [],
 ['l'],
 ['f'],
 ['e'],
 ['r', 'v', 'n', 'g'],
 ['r', 'u', 'n'],
 ['b', 'e', 'b', 'l', 'l'],
 ['p', 'r', 'c', 'c', 'e', '.'],
 ['S', 'e'],
 ['h', 'e', 'l', 'h'],
 ['e', 'x', 'p', 'e', 'r'],
 ['u', 'g', 'g', 'e'],
 ['h'],
 [],
 ['c', 'u', 'e'],
 ['n', 'c', 'r', 'e', 'e'],
 ['e', 'n', 'n'],
 ['n'],
 ['b', 'l'],
 ['p', 'r', 'e', 'u', 'r', 'e', '.'],
 ['I'],
 ['f', 'e', 'n'],
 ['f', 'e', 'e', 'l'],
 ['p', 'r', 'e', 'u', 'r', 'e'],
 ['p', 'e', 'r', 'f', 'r'],
 ['w', 'e', 'l', 'l'],
 [],
 ['c', 'h', 'l', ','],
 ['n', 'e', 'v', 'e', 'r'],
 ['e', 'e'],
 ['r', 'v', 'e'],
 [],
 ['b', 'e', 'e', 'r', '.'],
 ['H', 'e', 'l', 'h'],
 ['p', 'r', 'f', 'e', 'n', 'l'],
 [],
 ['b', 'r', 'c', 'l', 'l'],
 ['f', 'r'],
 ['u', 'r'],
 ['h', 'e', 'l', 'h', '.']]

In [24]:
print(frequency)

defaultdict(<class 'int'>, {'Brocolli': 1, 'is': 2, 'good': 3, 'to': 6, 'eat.': 1, 'My': 2, 'brother': 3, 'likes': 1, 'eat': 1, 'brocolli,': 1, 'but': 2, 'not': 1, 'my': 4, 'mother.': 1, 'mother': 2, 'spends': 1, 'a': 1, 'lot': 1, 'of': 1, 'time': 1, 'driving': 2, 'around': 1, 'baseball': 1, 'practice.': 1, 'Some': 1, 'health': 1, 'experts': 1, 'suggest': 1, 'that': 2, 'may': 1, 'cause': 1, 'increased': 1, 'tension': 1, 'and': 1, 'blood': 1, 'pressure.': 1, 'I': 1, 'often': 1, 'feel': 1, 'pressure': 1, 'perform': 1, 'well': 1, 'at': 1, 'school,': 1, 'never': 1, 'seems': 1, 'drive': 1, 'do': 1, 'better.': 1, 'Health': 1, 'professionals': 1, 'say': 1, 'brocolli': 1, 'for': 1, 'your': 1, 'health.': 1})


**7.3 Create the `corpora.Dictionary` object with the retained tokens.**

In [25]:
# Create Dictionary
id2word = corpora.Dictionary([frequency])

**7.4 Use the `dictionary.doc2bow()` function to convert the texts to bag-of-word representations.**

In [26]:
# Create Corpus
corpus = [id2word.doc2bow(text) for text in [frequency]]

**Why should we use this process?**

The main advantage is that this dictionary object has quick helper functions.

However, there are also some major performance advantages if you ever want to save your model to a file, then load it at a later time.  Tokenizations can take a while to be computed, especially when your text files are quite large. You can save these post-computed dictionary items to file, then load them from disk later which is quite a bit faster.  Also, it's possible to add new documents to your corpus without having to re-tokenize your entire set.  This is great for online systems that can take new documents on demmand.  

As you work with larger datasets with text, this is a much better way to handle LDA and other Gensim models from a performance point of view.

### 8. Set up the LDA model

We can create the gensim LDA model object like so:

```python
lda = models.LdaModel(
    # supply our sparse predictor matrix wrapped in a matutils.Sparse2Corpus object
    matutils.Sparse2Corpus(X, documents_columns=False),
    # or alternatively use the corpus object created with the dictionary in the previous frame!
    # corpus,
    # The number of topics we want:
    num_topics  =  3,
    # how many passes over the vocabulary:
    passes      =  20,
    # The id2word vocabulary we made ourselves
    id2word     =  vocab
    # or use the gensim dictionary object!
    # id2word     =  dictionary
)
```

In [27]:
lda = models.LdaModel(corpus=corpus, num_topics=5, passes=20, id2word=id2word)

### 9. Look at the topics

The model has a `.print_topics` function that accepts the number of topics to print and number of words per topic. The number before the word is the probability of occurance for that word in the topic.

In [28]:
for topics in lda.print_topics():
  print(topics[1])

0.018*"spends" + 0.018*"pressure" + 0.018*"say" + 0.018*"seems" + 0.018*"Health" + 0.018*"experts" + 0.018*"practice." + 0.018*"good" + 0.018*"perform" + 0.018*"eat"
0.018*"around" + 0.018*"spends" + 0.018*"brocolli," + 0.018*"suggest" + 0.018*"pressure" + 0.018*"is" + 0.018*"but" + 0.018*"blood" + 0.018*"experts" + 0.018*"a"
0.018*"better." + 0.018*"feel" + 0.018*"may" + 0.018*"around" + 0.018*"for" + 0.018*"school," + 0.018*"time" + 0.018*"that" + 0.018*"mother." + 0.018*"at"
0.018*"lot" + 0.018*"Brocolli" + 0.018*"brocolli" + 0.018*"brother" + 0.018*"say" + 0.018*"pressure" + 0.018*"cause" + 0.018*"My" + 0.018*"tension" + 0.018*"not"
0.018*"mother" + 0.018*"cause" + 0.018*"eat." + 0.018*"pressure." + 0.018*"My" + 0.018*"good" + 0.018*"health." + 0.018*"seems" + 0.018*"Some" + 0.018*"baseball"


In [ ]:
# There are 5 topics with 10 words each
# Each topic is combination of words and each word contributes a certain weightage to the topic
# Each word has same probability in each topic! 0.018...should add up to 1.0???

### 10. Get the topic scores for a document

The `.get_document_topics` function accepts a bag-of-words representation for a document and returns the scores for each topic.

In [30]:
for doc in documents:
    bow = id2word.doc2bow(doc.split())
    t = lda.get_document_topics(bow)

In [31]:
print(t)

[(0, 0.01820202), (1, 0.018202053), (2, 0.92719185), (3, 0.018202076), (4, 0.01820203)]


### 11. Label and visualize the topics

Lets come up with some high level labels. This is the subjective part of LDA. What do the word probabilties that represent topics mean?  Let's make some up.

Plot a heatmap of the topic probabilities for each of the documents.

In [17]:
# A:

### 12. Fit an LDA model with sklearn

Sklearn's LDA model is in the decomposition submodule:

```python
from sklearn.decomposition import LatentDirichletAllocation
```

One of the greatest benefits of the sklearn implementation is that it comes with the familiar `.fit()`, `.transform()` and `.fit_transform()` methods.

**12.1 Initialize and fit an sklearn LDA with `n_topics=3` on our output from the CountVectorizer.**

In [18]:
# A:

**12.2 Print out the topic-word distributions using the `.components_` attribute.**

Each row of this matrix represents a topic, and the columns are the words. (These are not probabilities).

In [19]:
# A:

**12.3 Use the `.transform()` method to convert the matrix into the topic scores.**

These are the document-topic distributions.

In [20]:
# A:

### 13. Further steps

This has been a very basic example.  LDA typically doesn't perform well on very small datasets.  You should try to see how it behaves on your own using a larger text dataset.  Keep in mind: finding the optimal number of topics can be tricky and subjective.

**Generally, you should consider:**
- How well topics are applied to documents overall
- The strength of topics overall, to all documents
- Improving preprocessing such as stopword removal
- Building a nice web interface to explore your documents (see: [LDAExplorer](https://github.com/dyerrington/LDAExplorer), and [pyLDAvis](https://github.com/bmabey/pyLDAvis/blob/master/README.rst))

These general guidelines should help you tune your hyperparameter **K** for number of topics.